# Create reports using the ArcGIS API for Python

<div class="alert alert-block alert-info">This notebook uses the ArcGIS API for Python. For more information, see the <a href="https://developers.arcgis.com/python/" target=_blank>ArcGIS API for Python documentation and guides</a>.</div>

ArcGIS Survey123 reports are an effective way to quickly visualize, interpret, and share survey responses. Reports use a template you can personalize, that's applied to your survey results and shared as either a Microsoft Word or PDF document.

There are several ways to create reports from your survey data. The most common method is through the **Report** panel on the Data tab for your survey on the Survey123 website. Here, you can manage report templates and generate reports in PDF and DOCX format.

You can also use a webhook to trigger a process to create and share reports that contain survey results. A webhook is useful if want the report to be triggered by an action. For example, each time a survey is submitted, a webhook can trigger the **Create report** module in Integromat to generate a report. For more information, see <a href="https://doc.arcgis.com/en/survey123/browser/create-surveys/webhooks.htm" target="_blank">Webhooks</a>.

Another approach is to run an automated reporting process at regular intervals. For example, you might want to produce a report for all surveys submitted in the past week, month, or quarter. This is where the ArcGIS Survey123 module in the ArcGIS API for Python can be useful: you can set up a script and run it periodically as a scheduled task.

This sample notebook demonstrates ways you can use the ArcGIS Survey123 module to work with reports and covers the following functionality: 

* <a href="#identify-report-templates">Identify report templates associated with a survey</a>
* <a href="#generate-default-template">Generate a default report template</a>
* <a href="#check-template-syntax">Check template syntax</a>
* <a href="#upload-report-template">Associate a report template with a survey</a>
* <a href="#estimate-credits">Estimate credits</a>
* <a href="#create-sample-report">Create sample report</a>
* <a href="#generate-a-single-report">Generate a single report</a>
* <a href="#generate-pdf-report">Generate a report as a PDF</a>
* <a href="#generate-multiple-reports">Generate multiple reports</a>
* <a href="#generate-multiple-reports-org">Generate multiple reports and save to your organization</a>
* <a href="#all-parameter-report">Generate a report with all possible parameters</a>
* <a href="#update-report-template">Update report template</a>
* <a href="#list-saved-reports">List saved reports</a>

<div class="alert alert-block alert-warning">Printing of reports is an ArcGIS Online Premium Service that consumes credits. Reports are charged at a minimum cost of 0.5 credits. For more information, see <a href="https://doc.arcgis.com/en/arcgis-online/administer/credits.htm" target=_blank>Understand credits</a>.</div>

The first step is to connect to your GIS.

In [1]:
import arcgis
from arcgis.gis import GIS
gis = GIS("home")

Next, a Survey Manager is defined, and a survey is accessed using the form's item ID. A survey in the Survey Manager is a single instance of a survey project that contains the item information and properties and provides access to the underlying survey dataset. For more information on Survey Manager, see the <a href="https://developers.arcgis.com/python/api-reference/arcgis.apps.survey123.html#arcgis.apps.survey123.SurveyManager" target="_blank">API Reference for the ArcGIS API for Python</a>.

In [2]:
survey_manager = arcgis.apps.survey123.SurveyManager(gis)
survey_by_id = survey_manager.get("e7709174ba48426c880e504e11319970")
print(survey_by_id)

<Survey @ Water Quality Inspection>


<a id='identify-report-templates'></a>
### Identify report templates associated with a survey

The `report_templates` method returns a list of all report templates already associated with the survey. For each template in the list the title is printed using the print statement below. 

In [3]:
templates = survey_by_id.report_templates
p = [print(t.title) for t in templates]

Water Quality Inspection


<a id='generate-default-template'></a>
### Generate a default report template

In the case where no report templates are associated with the survey, a default template can be generated using the `create_report_template` method. The `create_report_template` method creates a simple default template and downloads it to the `Temp` folder.

In [4]:
new_template = survey_by_id.create_report_template()
print(new_template)

\Temp\template_30398


<a id='check-template-syntax'></a>
### Check template syntax

The `check_template_syntax` method checks the report template and identifies any syntax issues that could lead to a failure when generating a report. The method requires a directory path to a template file. The output string indicates if the check was successful or not. If the check is unsuccessful, the output will also include an error message.

In [5]:
check = survey_by_id.check_template_syntax(new_template)
print("Result of template syntax check:", check)

Result of template syntax check: {'success': True, 'individualSectionCount': 1, 'summarySectionCount': 0, 'details': []}


<a id='upload-report-template'></a>
### Associate a report template with a survey

The `upload_report_template` method associates a report template with a survey. First, the method checks the syntax in the template file. If the check is successful, the template is uploaded to your ArcGIS organization in the same folder as the survey and linked with the survey for use in reports. If the check is unsuccessful an error message is returned. The method takes one required parameter, `template_file` which is the directory path to the template file, and an optional parameter `template_name` which will be used as the item name when uploaded to your organization. If `template_name` is omitted, the name of the template file will be used as the name of the item.

In [6]:
upload_template = survey_by_id.upload_report_template(template_file=new_template, template_name="PythonAPITemplate")
print(upload_template)
templates = survey_by_id.report_templates
p = [print(t.title) for t in templates]

<Item title:"PythonAPITemplate" type:Microsoft Word owner:NinjaGreen>
Water Quality Inspection
PythonAPITemplate


<a id='estimate-credits'></a>
### Estimate credits

For ArcGIS Online users, the number of credits that will be consumed by running a report with specific parameters can be estimated using the `estimate()` method. The method takes two parameters, the `report_template` item and a `where` clause.

In [7]:
credits = survey_by_id.estimate(templates[1], where="1=1")
print("The estimated amount of credits are: ", credits)

The estimated amount of credits are:  {'success': True, 'resultInfo': {'totalRecords': 189, 'cost': 94.5}}


<a id='create-sample-report'></a>
### Create a sample report

You can create a watermarked sample PDF report using the `create_sample_report` method. This method is similar to the `generate_report` method but it does not consume credits, making it useful for testing and refinement of your report template before you generate any formal reports.

The `create_sample_report` method requires a template item as a parameter and can include the following optional parameters:

- `where`—Specify which records to include in the report.
- `utc_offset`—Adjust dates and times to match a specific time zone.
- `report_title`—Specify the file name (without extension) of the output report file.
- `merge_files`—Use this parameter when multiple records are to be included in a single report file. The `merge_files` parameter accepts the following values:
  - `none`—Print multiple records in split mode, where each record becomes a separated report file. This is the default value.
  - `nextPage`—Print multiple records in merge mode, where the content of the next record starts on a new page.
  - `continuous`—Print multiple records in merge mode, where the content of the next record starts on the same page as the previous record.
- `survey_item`—Provides additional information on survey structure. For example, Multiple Choice questions require this parameter in order to access information on list labels..
- `webmap_item`—Specify the basemap to use for geometry questions.
- `map_scale`—Specify the scale of the map used for geometry questions.
- `locale`—Apply a regional format to numbers, dates, and times.

<div class="alert alert-block alert-info">Sample reports can print a maximum of 20 results at a time.</div>

In [8]:
webmap = gis.content.search(query="title:Water Quality Inspection Web Map", item_type="Web Map")
wm_item=webmap[0]
sample = survey_by_id.create_sample_report(templates[1], where="objectid=1", utc_offset="-07:00", 
                                           report_title="Sample_Report", merge_files="none", survey_item=survey_by_id, 
                                           webmap_item=wm_item, map_scale="10000", locale='en')
print("Sample Report: ", sample)

Sample Report:  \Temp\Sample_Report_sample.pdf


<a id='generate-a-single-report'></a>
### Generate a single report

Use the `generate_report` method to generate a single report by specifying an object ID in the `where` parameter. In the cell below, the index of 1 specifies the report template uploaded earlier in this notebook as the report template to use when generating the report.

The `where` clause follows the same format as the `where` parameter when <a href="https://developers.arcgis.com/rest/services-reference/query-feature-service-layer-.htm" target="_blank">querying a feature service layer</a>.  

In [9]:
report = survey_by_id.generate_report(templates[1], where="objectid=122")
print(report)

\Temp\Water%20Quality%20Inspection_OID122.docx


<a id='generate-pdf-report'></a>
### Generate a report as a PDF

Specify `pdf` as the `output_format` to print the report as a PDF document. If the `output_format` parameter is omitted the report will default to a Microsoft Word DOCX file.

In [10]:
pdf_report = survey_by_id.generate_report(templates[1], where="objectid=1", output_format="pdf")
print("Single report as PDF: ", pdf_report)

Single report as PDF:  \Temp\Water%20Quality%20Inspection_OID1.pdf


<a id='generate-multiple-reports'></a>
### Generate multiple reports

To generate reports for more than one survey record, specify multiple object IDs in the `where` clause, for example, `"objectid=1 OR objectid=2"`. Alternatively, omit the `where` clause to generate reports for all features in the feature layer. In both scenarios the output is saved as a ZIP file that will be stored locally on disk when run from a Python notebook.

In the example below, the `where` clause queries the feature layer to return records where a water station advisory was enacted after January 1, 2020. Any records that match the query will be included in the report. You can create a dynamic `where` clause that can query records for the last week, month, or year for example as well as add additional criteria from the data to create your reports. 

In [11]:
local_batch_reports = survey_by_id.generate_report(templates[1], where="ws_advisory = 'Yes' and ws_advisory_start_date > '01/01/2020'", 
                                                   report_title="SingleReportInstance", package_name="ReportPackageNamePython")
print("Local batch reports: ", local_batch_reports)

Local batch reports:  \Temp\ReportPackageNamePython.zip


<a id='generate-multiple-reports-org'></a>
### Generate multiple reports and save to your organization

To generate reports and save them in your ArcGIS organization content, add the `folder_id` argument. By specifying a `folder_id` the reports are saved in your content as a code sample, which is available for download as a ZIP file.

The code sample can then be shared to a group or publicly so that others can download the ZIP and view the reports.

In [12]:
import datetime
from datetime import date, timedelta
nowstring = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
prev_days = date.today() - timedelta(7)
folder_ID = survey_by_id.properties['ownerFolder']
org_batch_reports = survey_by_id.generate_report(templates[1], where="CreationDate > '{}'".format(prev_days),
                                                 package_name="Test_{0}".format(nowstring), folder_id=folder_ID)
print("Org batch reports: ", org_batch_reports)
search_for_batch_reports = gis.content.search("Test_{0}".format(nowstring))
print("Search result: ", search_for_batch_reports)

Org batch reports:  <Item title:"Test_20210817132201" type:Code Sample owner:NinjaGreen>
Search result:  [<Item title:"Test_20210817132201" type:Code Sample owner:NinjaGreen>]


<a id='all-parameter-report'></a>
### Generate a report with all possible parameters

The example below demonstrates all possible parameters that can be used in the `generate_report` method. Most of these parameters are listed in the description of the `create_sample_report` method, covered earlier in this notebook. Additional parameters that can be used with the `generate_report` method are as follows:

- `package_name`—Define the name of the ZIP file or code sample that is created if multiple reports are generated.
- `output_format`—Specify the file format for the report output. Accepted values are `docx` and `pdf`.
- `folder_id`—Upload the report to a specific folder in your ArcGIS organization.

In [13]:
folder_ID = survey_by_id.properties['ownerFolder']
webmap = gis.content.search(query="title:Water Quality Inspection Web Map", item_type="Web Map")
wm_item=webmap[0]
all_params = survey_by_id.generate_report(templates[1], where="ws_advisory = 'Yes' and ws_advisory_start_date > '01/01/2020'", utc_offset="-07:00", report_title="All_Params_Report",
                                          package_name="All_Params_Package", output_format="pdf", folder_id=folder_ID, 
                                          merge_files="none", survey_item=survey_by_id, webmap_item=wm_item, 
                                          map_scale="10000", locale="en")
print("All params: ", all_params)

All params:  <Item title:"All_Params_Package" type:Code Sample owner:NinjaGreen>


<a id='update-report-template'></a>
### Update report template

Report templates may need to be updated to incorporate changes to a survey, or to modify the page layout, text formatting, and so on.

Use the `update_report_template` method to programmatically update existing report templates associated with a survey. This is particularly useful if you need to bulk update the templates for various surveys in your organization. The `update_report_template` method first checks the syntax of the updated template file. If the check is successful, the method updates the existing item in your organization with the new template. If the check is unsuccessful an error message is returned.

In [14]:
updated_template = r"\Temp\PythonAPITemplate.docx"
update = survey_by_id.update_report_template(updated_template)
print(update)

[<Item title:"PythonAPITemplate" type:Microsoft Word owner:NinjaGreen>]


<a id='list-saved-reports'></a>
### List saved reports

Finally, use the `reports` property to list the reports stored in the ArcGIS organizational account for the current GIS connection. This will return all reports that are saved in that named user's content.

In [15]:
recentReports = survey_by_id.reports
p = [print(t.title) for t in recentReports]

Manhole_Inspection_report_7844e0
Test_summary1
UC2020 Hydrants Maintenance_OID_summary1
UC2020 Hydrants Maintenance_5 records_20201211002634
Daily Fish Log_sampleTemplateSummaryIndividual
Manhole Inspection sample template
